In [0]:
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
!pip install fastai

Looking in links: https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
     |████████████████████████████████| 704.8MB 25kB/s 


In [0]:
import fastai
from fastai import *
from fastai.text import * 
import pandas as pd
import numpy as np
from functools import partial
import io
import os

In [0]:
path = untar_data(URLs.IMDB_SAMPLE)


In [5]:
df = pd.read_csv(path/'texts.csv')
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


In [0]:
# does preprocessing
# Language model data
data_lm = TextLMDataBunch.from_csv(path, 'texts.csv')
# Classifier model data
data_clas = TextClasDataBunch.from_csv(path, 'texts.csv', vocab=data_lm.train_ds.vocab, bs=64)

In [9]:
print(data_lm.__dict__)

{'dl_tfms': [], 'device': device(type='cuda'), 'train_dl': DeviceDataLoader(dl=<torch.utils.data.dataloader.DataLoader object at 0x7fb21c190208>, device=device(type='cuda'), tfms=[], collate_fn=<function data_collate at 0x7fb27ffe3bf8>), 'valid_dl': DeviceDataLoader(dl=<torch.utils.data.dataloader.DataLoader object at 0x7fb21c190400>, device=device(type='cuda'), tfms=[], collate_fn=<function data_collate at 0x7fb27ffe3bf8>), 'fix_dl': DeviceDataLoader(dl=<torch.utils.data.dataloader.DataLoader object at 0x7fb21c190668>, device=device(type='cuda'), tfms=[], collate_fn=<function data_collate at 0x7fb27ffe3bf8>), 'test_dl': None, 'single_dl': DeviceDataLoader(dl=<torch.utils.data.dataloader.DataLoader object at 0x7fb21c1907b8>, device=device(type='cuda'), tfms=[], collate_fn=<function data_collate at 0x7fb27ffe3bf8>), 'path': PosixPath('/root/.fastai/data/imdb_sample'), 'label_list': LabelLists;

Train: LabelList (799 items)
x: LMTextList
xxbos good movie , good music , good background an

In [0]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

In [0]:
data_lm = load_data(path, 'data_lm_export.pkl')
data_clas = load_data(path, 'data_clas_export.pkl', bs=64)

In [18]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.7)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.397957,3.940933,0.281059,00:05


In [19]:
learn.unfreeze()
learn.fit_one_cycle(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.023118,3.878031,0.288457,00:07
1,3.920570,3.852451,0.288744,00:07
2,3.742456,3.833615,0.291406,00:07
3,3.553226,3.843387,0.290099,00:07
4,3.415928,3.846281,0.290418,00:07


In [20]:
learn.predict("This is a review about", n_words=20)

'This is a review about the flaws and subject of compare friendship , truth and decency each another in a film and on a manga'

In [0]:
learn.save_encoder('ft_enc')

In [26]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (799 items)
x: TextList
xxbos xxup in xxup cold xxup blood is masterfully directed and adapted by xxmaj richard xxmaj brooks . xxmaj however , it 's also so bent on being realistic , it 's sometimes more xxunk than entertaining . xxmaj recounting the brutal killing of a xxmaj xxunk family , author xxmaj xxunk xxmaj capote focused on xxunk , xxunk himself and the reader up in the subject xxup and subjects ! xxmaj brooks xxunk wildly from that approach in favor of something closer to xxunk . xxmaj although he films on actual locations , he keeps his distance . xxmaj the murderers are portrayed as xxunk imbeciles , which surely they were . xxmaj they 're not seen as xxunk souls ( as in the xxmaj capote book ) and the xxunk of their act is xxunk blunt . xxmaj scott xxmaj wilson and xxmaj robert xxmaj blake are excellent as the killers as is the supporting cast , including xxmaj john xxmaj xxunk and xxmaj paul xxmaj stewart as the reporte

In [33]:
data_clas.show_batch()

text,target
"xxbos xxmaj raising xxmaj victor xxmaj vargas : a xxmaj review \n \n xxmaj you know , xxmaj raising xxmaj victor xxmaj vargas is like sticking your hands into a big , steaming bowl of xxunk . xxmaj it 's warm and gooey , but you 're not sure if it feels right . xxmaj try as i might , no matter how warm and gooey xxmaj raising xxmaj",negative
"xxbos xxup the xxup shop xxup around xxup the xxup corner is one of the sweetest and most feel - good romantic comedies ever made . xxmaj there 's just no getting around that , and it 's hard to actually put one 's feeling for this film into words . xxmaj it 's not one of those films that tries too hard , nor does it come up with",positive
"xxbos xxmaj this film sat on my xxmaj tivo for weeks before i watched it . i dreaded a self - indulgent xxunk flick about relationships gone bad . i was wrong ; this was an xxunk xxunk into the screwed - up xxunk of xxmaj new xxmaj yorkers . \n \n xxmaj the format is the same as xxmaj max xxmaj xxunk ' "" xxmaj la xxmaj ronde",positive
"xxbos xxmaj many xxunk that this is n't just a classic due to the fact that it 's the first xxup 3d game , or even the first xxunk - up . xxmaj it 's also one of the first xxunk games , one of the xxunk definitely the first ) truly claustrophobic games , and just a pretty well - xxunk gaming experience in general . xxmaj with graphics",positive
"xxbos i really wanted to love this show . i truly , honestly did . \n \n xxmaj for the first time , gay viewers get their own version of the "" xxmaj the xxmaj bachelor "" . xxmaj with the help of his obligatory "" hag "" xxmaj xxunk , xxmaj james , a good looking , well - to - do thirty - something has the chance",negative


In [27]:
learn.fit_one_cycle(5, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.664245,0.596720,0.706468,00:04
1,0.539121,0.514689,0.805970,00:04
2,0.461827,0.439611,0.830846,00:04
3,0.420920,0.410348,0.840796,00:04
4,0.378344,0.408732,0.840796,00:04


In [29]:
learn.freeze_to(-2)
learn.fit_one_cycle(3, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.316739,0.475275,0.800995,00:05
1,0.286474,0.392188,0.875622,00:05
2,0.238910,0.413846,0.880597,00:05


In [30]:
learn.unfreeze()
learn.fit_one_cycle(3, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.189874,0.374581,0.855721,00:08
1,0.158524,0.406615,0.865672,00:08
2,0.138124,0.403244,0.845771,00:09


In [69]:
learn.predict("But I really liked the ending. bad movie. ")

(Category negative, tensor(0), tensor([0.9661, 0.0339]))

In [58]:
# get predictions
preds, targets = learn.get_preds()

predictions = np.argmax(preds, axis = 1)
print(pd.crosstab(predictions, targets))

from sklearn.metrics import accuracy_score

score = accuracy_score(predictions, targets)
print(score)

col_0   0   1
row_0        
0      92  15
1      16  78
0.845771144278607
